In [ ]:
import json
import os
import shutil
import requests
import cv2
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
# Step 2: Define paths for images and annotations
json_dirs = {
    'train': '/content/drive/MyDrive/dataset_us/train/annotations',
    'val': '/content/drive/MyDrive/dataset_us/val/annotations'
}
image_dirs = {
    'train': '/content/drive/MyDrive/dataset_us/train/images',
    'val': '/content/drive/MyDrive/dataset_us/val/images'
}
output_dirs = {
    'train': '/content/drive/MyDrive/dataset_us/output/yolo/train',
    'val': '/content/drive/MyDrive/dataset_us/output/yolo/val'
}

# Create output directories if they don't exist
os.makedirs(output_dirs['train'], exist_ok=True)
os.makedirs(output_dirs['val'], exist_ok=True)


In [ ]:
# Define the classes based on the provided JSON files
classes = ['First_Name', 'Last_Name', 'DLNumber', 'Address', 'DOB', 'ExpiryDate', 'IssueDate', 'Sex', 'Class', 'Name']

# Step 3: Function to convert JSON annotations to YOLO format
def convert_to_yolo_format(json_path, image_path, output_dir, classes):
    with open(json_path, 'r') as f:
        data = json.load(f)

    image_width = data['image_size']['width']
    image_height = data['image_size']['height']

    yolo_annotations = []
    for item in data['annotations']:
        if item['label'] in classes:
            class_id = classes.index(item['label'])
            xmin = item['coordinates']['xmin']
            ymin = item['coordinates']['ymin']
            xmax = item['coordinates']['xmax']
            ymax = item['coordinates']['ymax']

            # Convert to YOLO format
            x_center = (xmin + xmax) / 2 / image_width
            y_center = (ymin + ymax) / 2 / image_height
            width = (xmax - xmin) / image_width
            height = (ymax - ymin) / image_height

            yolo_annotations.append(f"{class_id} {x_center} {y_center} {width} {height}")

    # Save to output directory
    base_name = os.path.splitext(os.path.basename(json_path))[0]
    yolo_file_path = os.path.join(output_dir, base_name + '.txt')
    with open(yolo_file_path, 'w') as f:
        f.write('\n'.join(yolo_annotations))

    # Copy image to output directory
    image_output_path = os.path.join(output_dir, os.path.basename(image_path))
    if not os.path.exists(image_output_path):
        shutil.copy(image_path, output_dir)

In [ ]:
# Step 4: Convert annotations for training and validation sets
for phase in ['train', 'val']:
    json_dir = json_dirs[phase]
    image_dir = image_dirs[phase]
    output_dir = output_dirs[phase]

    for json_file in os.listdir(json_dir):
        if json_file.endswith('.json'):
            json_file_path = os.path.join(json_dir, json_file)
            image_file = json_file.replace('.json', '.jpg')  # Adjust the extension if necessary
            image_file_path = os.path.join(image_dir, image_file)
            if os.path.exists(image_file_path):
                convert_to_yolo_format(json_file_path, image_file_path, output_dir, classes)
            else:
                print(f"Image file not found: {image_file_path}")